In [2]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from typing_extensions import TypedDict, Annotated
from langgraph.graph import START, StateGraph, END
from langchain_community.document_loaders import WebBaseLoader

import time


In [3]:
# To Handle: https://langchain-ai.github.io/langgraph/troubleshooting/errors/INVALID_CONCURRENT_GRAPH_UPDATE/
def custom_reducer(obj1, obj2):
  return obj1 if obj1 else obj2


In [4]:
class SharedState(TypedDict):
    github_handle: Annotated[str, custom_reducer]
    medium_handle: Annotated[str, custom_reducer]

    github_profile_content: Annotated[str, custom_reducer]
    medium_profile_content: Annotated[str, custom_reducer]

    github_profile_summary: Annotated[str, custom_reducer]
    medium_profile_summary: Annotated[str, custom_reducer]
    
    model: Annotated[ChatOpenAI, custom_reducer]

In [5]:
def build_model(shared_state: SharedState) -> SharedState:
    model = ChatOpenAI(model= "gpt-5")
    shared_state['model'] = model

    return shared_state

In [6]:
def pull_github_profile_content(shared_state: SharedState) -> SharedState:
    '''Pulls the GitHub profile content for the given GitHub handle.'''
    print('Pulling GitHub profile content...')
    github_profile_url = 'https://www.github.com/' + shared_state['github_handle']
    documents = WebBaseLoader(github_profile_url).load()
    page_content = ''

    for document in documents:
        page_content += document.page_content
    
    shared_state['github_profile_content'] = page_content.strip()
    return shared_state

In [7]:
def pull_medium_profile_content(shared_state: SharedState) -> SharedState:
    '''Pulls the Medium profile content for the given Medium handle.'''
    print('Pulling Medium profile content...')
    medium_profile_url = f'https://{shared_state['medium_handle']}.medium.com'
    documents = WebBaseLoader(medium_profile_url).load()
    page_content = ''

    for document in documents:
        page_content += document.page_content
    
    shared_state['medium_profile_content'] = page_content.strip()
    return shared_state

In [8]:
def summarize_github_profile(shared_state: SharedState) -> SharedState:
    '''Summarizes the GitHub profile content.'''
    print('Summarizing GitHub profile content...')
    model = shared_state['model']
    github_profile_content = shared_state['github_profile_content']

    prompt = f'''Summarize the following GitHub profile content in a concise manner
    and highlight name, organization, followers, location, contact information key skills, 
    projects, and contributions.

    Github Profile Content: {github_profile_content}
    '''
    response = model.invoke(prompt)
    shared_state['github_profile_summary'] = response.content.strip()

    return shared_state


In [9]:
def summarize_medium_profile(shared_state: SharedState) -> SharedState:
    '''Summarizes the Medium profile content.'''
    print('Summarizing Medium profile content...')
    model = shared_state['model']
    medium_profile_content = shared_state['medium_profile_content']

    prompt = f'''Summarize the following Medium profile content in a concise manner
    and highlight name, followers, posted articles, and description.

    Medium Profile Content: {medium_profile_content}
    '''
    response = model.invoke(prompt)
    shared_state['medium_profile_summary'] = response.content.strip()

    return shared_state


In [12]:
def build_parallel_graph():
  # Building a Graph
  # State of the Graph that will be shared among nodes.
  workflow = StateGraph(SharedState)

  # Add nodes.
  workflow.add_node("build_model", build_model)
  workflow.add_node("pull_github_profile_content", pull_github_profile_content)
  workflow.add_node("pull_medium_profile_content", pull_medium_profile_content)
  workflow.add_node("summarize_github_profile", summarize_github_profile)
  workflow.add_node("summarize_medium_profile", summarize_medium_profile)

  # Define the edges of the graph.
  workflow.add_edge(START, "build_model")
  workflow.add_edge("build_model", "pull_github_profile_content")
  workflow.add_edge("build_model", "pull_medium_profile_content")
  workflow.add_edge("pull_github_profile_content", "summarize_github_profile")
  workflow.add_edge("pull_medium_profile_content", "summarize_medium_profile")
  workflow.add_edge("summarize_medium_profile", END)
  workflow.add_edge("summarize_github_profile", END)

  graph = workflow.compile()

  return graph

In [13]:
load_dotenv()
start_time = time.perf_counter()

compiled_graph = build_parallel_graph()
response = compiled_graph.invoke({
    'github_handle': 'SauravP97',
    'medium_handle': 'srvptk97',
})

print('\n\n' + response['github_profile_summary'])
print('\n\n' + response['medium_profile_summary'])

end_time = time.perf_counter()

print(f'\n\nExecution Time: {end_time - start_time:.2f} seconds')

Pulling Medium profile content...Pulling GitHub profile content...

Summarizing GitHub profile content...Summarizing Medium profile content...



Name: Saurav Prateek
Organization: Google
Followers: 166
Location: Gurugram, India
Contact: LinkedIn profile - https://www.linkedin.com/in/saurav-prateek-7b2096140/
Key Skills: Web Solutions Engineering, Software Development
Projects: 
- Saurav-s-DSA-Templates
- Saurav-Low-Level-Design-Template
- AI-Engineering-101
- LangPost
- micrograd-java
- makemore
Contributions: Systems That Scale (Engineering Newsletter), YouTube channel (tech-related content)


Name: Saurav Prateek
Followers: 91
Posted Articles:
- Dynamic Programming (DP) on Trees | Solving Catering Contracts Problem Statement
- My Google Interview Experience | Web Solutions Engineer role
- Principles of Clean Coding — Objects, Error Handling and Clean Integration (Part 2)
- Principles of Clean Coding — Every Entry Level Developer should Follow (Part 1)
- I am writing a Library for Qu

In [14]:
def build_serial_graph():
  # Building a Graph
  # State of the Graph that will be shared among nodes.
  workflow = StateGraph(SharedState)

  # Add nodes.
  workflow.add_node("build_model", build_model)
  workflow.add_node("pull_github_profile_content", pull_github_profile_content)
  workflow.add_node("pull_medium_profile_content", pull_medium_profile_content)
  workflow.add_node("summarize_github_profile", summarize_github_profile)
  workflow.add_node("summarize_medium_profile", summarize_medium_profile)

  # Define the edges of the graph.
  workflow.add_edge(START, "build_model")
  workflow.add_edge("build_model", "pull_github_profile_content")
  workflow.add_edge("pull_github_profile_content", "summarize_github_profile")
  workflow.add_edge("summarize_github_profile", "pull_medium_profile_content")
  workflow.add_edge("pull_medium_profile_content", "summarize_medium_profile")
  workflow.add_edge("summarize_medium_profile", END)

  graph = workflow.compile()

  return graph

In [15]:
start_time = time.perf_counter()

compiled_graph = build_serial_graph()
response = compiled_graph.invoke({
    'github_handle': 'SauravP97',
    'medium_handle': 'srvptk97',
})

print('\n\n' + response['github_profile_summary'])
print('\n\n' + response['medium_profile_summary'])

end_time = time.perf_counter()

print(f'\n\nExecution Time: {end_time - start_time:.2f} seconds')

Pulling GitHub profile content...
Summarizing GitHub profile content...
Pulling Medium profile content...
Summarizing Medium profile content...


Name: Saurav Prateek
Organization: Google
Followers: 166
Location: Gurugram, India
Contact: https://www.linkedin.com/in/saurav-prateek-7b2096140/
Key Skills: Web Solutions, Software Engineering
Projects: Saurav-s-DSA-Templates, Saurav-Low-Level-Design-Template, AI-Engineering-101, LangPost, micrograd-java, makemore
Contributions: Web Solutions Engineer II at Google, previously Software Engineer at GeeksForGeeks, maintained an Engineering Newsletter "Systems That Scale," YouTube channel with tech-related content.


Name: Saurav Prateek
Followers: 93
Articles:
1. Dynamic Programming (DP) on Trees | Solving Catering Contracts Problem Statement
2. My Google Interview Experience | Web Solutions Engineer role
3. Principles of Clean Coding — Objects, Error Handling and Clean Integration (Part 2)
4. Principles of Clean Coding — Every Entry Level Deve

In [ ]:
print(build_parallel_graph().get_graph().draw_mermaid())

In [ ]:
print(build_serial_graph().get_graph().draw_mermaid())